# Bab 4: Melatih Model (Training Models)

### 1. Pendahuluan

Bab 4 ini beranjak dari pengenalan model *machine learning* sebagai "kotak hitam" dan mulai menyelami mekanisme internal serta algoritma pelatihan. Pemahaman mendalam tentang cara kerja model ini penting untuk memilih algoritma yang tepat, menyetel *hyperparameter*, melakukan *debugging*, dan analisis kesalahan secara efisien. Konsep-konsep yang dibahas dalam bab ini juga merupakan dasar penting untuk memahami *neural network*.

Bab ini secara khusus membahas:
* Model Regresi Linier dan dua metode pelatihannya: *Normal Equation* (solusi *closed-form*) dan *Gradient Descent* (pendekatan iteratif).
* Regresi Polinomial untuk data non-linier, termasuk deteksi *overfitting* menggunakan *learning curves* dan teknik regularisasi.
* Dua model klasifikasi: Regresi Logistik dan Regresi Softmax.

---

### 2. Regresi Linier (Linear Regression)

Model regresi linier membuat prediksi dengan menghitung jumlah bobot (*weighted sum*) dari fitur-fitur input, ditambah dengan istilah bias (*bias term*).

Rumus model Regresi Linier adalah sebagai berikut:
$$\hat{y} = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \cdots + \theta_n x_n$$
Dalam bentuk vektor, rumus ini menjadi:
$$\hat{y} = h_{\boldsymbol{\theta}}(\mathbf{x}) = \boldsymbol{\theta} \cdot \mathbf{x}$$
di mana:
* $\hat{y}$ adalah nilai yang diprediksi.
* $n$ adalah jumlah fitur.
* $x_i$ adalah nilai fitur ke-$i$.
* $\theta_j$ adalah parameter model ke-$j$ (termasuk istilah bias $\theta_0$ dan bobot fitur $\theta_1, \theta_2, \ldots, \theta_n$).
* $\boldsymbol{\theta}$ adalah vektor parameter model.
* $\mathbf{x}$ adalah vektor fitur *instance* (dengan $x_0$ selalu sama dengan 1).

Fungsi biaya MSE untuk model Regresi Linier adalah:
$$\text{MSE}(\mathbf{X}, h_{\boldsymbol{\theta}}) = \frac{1}{m} \sum_{i=1}^{m} (\boldsymbol{\theta}^\intercal \mathbf{x}^{(i)} - y^{(i)})^2$$

#### a. The Normal Equation
*Normal Equation* adalah solusi matematis (*closed-form*) yang secara langsung menghitung nilai $\boldsymbol{\theta}$ yang meminimalkan fungsi biaya.
$$\boldsymbol{\theta} = (\mathbf{X}^\intercal \mathbf{X})^{-1} \mathbf{X}^\intercal \mathbf{y}$$

Berikut adalah implementasi menggunakan NumPy dan Scikit-Learn.
```python
import numpy as np
import matplotlib.pyplot as plt

# Membuat data linier acak
X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)

# Menambahkan x0 = 1 ke setiap instance (untuk bias term)
X_b = np.c_[np.ones((100, 1)), X]

# Menghitung theta terbaik menggunakan Normal Equation
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

print("Theta terbaik yang dihitung dengan Normal Equation:")
print(theta_best)

# Membuat prediksi menggunakan theta yang ditemukan
X_new = np.array([[0], [2]])
X_new_b = np.c_[np.ones((2, 1)), X_new]
y_predict = X_new_b.dot(theta_best)

print("\nPrediksi untuk X_new:")
print(y_predict)

# Plot data dan garis regresi
plt.plot(X, y, "b.")
plt.plot(X_new, y_predict, "r-", label="Prediksi")
plt.xlabel("<span class="math-inline">x\_1</span>")
plt.ylabel("<span class="math-inline">y</span>")
plt.axis([0, 2, 0, 15])
plt.legend()
plt.title("Regresi Linier dengan Normal Equation")
plt.show()

# Melakukan hal yang sama dengan Scikit-Learn
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X, y)
print("\nScikit-Learn intercept dan coefficient:")
print(lin_reg.intercept_, lin_reg.coef_)
```

### b. Computational Complexity

* Normal Equation: Memiliki kompleksitas komputasi sekitar $O(n^{2.4})$ hingga $O(n^3)$. Ini menjadi sangat lambat ketika jumlah fitur ($n$) sangat besar.
* SVD (Scikit-Learn's LinearRegression): Sekitar $O(n^2)$.
* Prediksi sangat cepat setelah model dilatih.

---

### 3. Gradient Descent

Gradient Descent (GD) adalah algoritma optimisasi yang secara iteratif menyesuaikan parameter untuk meminimalkan fungsi biaya.
* Learning Rate (η): Menentukan ukuran langkah. Jika terlalu kecil, konvergensi akan lambat. Jika terlalu besar, bisa divergen.
* Feature Scaling: Penting untuk memastikan semua fitur memiliki skala yang serupa agar konvergensi lebih cepat.

Langkah Gradient Descent:

$$\boldsymbol{\theta}{next step} = \boldsymbol{\theta} - \eta \nabla{\boldsymbol{\theta}} \text{MSE}(\boldsymbol{\theta})$$

### a. Batch Gradient Descent

Algoritma ini menggunakan seluruh training set untuk menghitung gradient pada setiap langkah.

```python
eta = 0.1  # learning rate
n_iterations = 1000
m = 100 # jumlah instance

theta = np.random.randn(2, 1)  # inisialisasi acak

for iteration in range(n_iterations):
    gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
    theta = theta - eta * gradients

print("Theta terbaik yang dihitung dengan Batch Gradient Descent:")
print(theta)
```

### b. Stochastic Gradient Descent

Stochastic Gradient Descent (SGD) mengambil satu instance acak pada setiap langkah. Ini jauh lebih cepat tetapi hasilnya "memantul" di sekitar minimum. Untuk mengatasinya, learning rate dikurangi secara bertahap (learning schedule).

### c. Mini-batch Gradient Descent

Algoritma ini menghitung gradient pada kelompok kecil (mini-batches). Ini menawarkan kompromi antara kecepatan SGD dan kestabilan Batch GD.

---

### 4. Regresi Polinomial (Polynomial Regression)

Memungkinkan model linier untuk menyesuaikan data non-linier dengan menambahkan pangkat dari setiap fitur sebagai fitur baru.

```python
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# Membuat data non-linier
m = 100
X = 6 * np.random.rand(m, 1) - 3
y = 0.5 * X**2 + X + 2 + np.random.randn(m, 1)

# Menambahkan fitur polinomial (derajat 2)
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly_features.fit_transform(X)

# Melatih model Linear Regression pada data yang sudah ditransformasi
lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)

print("Intercept dan Coefficient dari Regresi Polinomial:")
print(lin_reg.intercept_, lin_reg.coef_)

# Plot hasil
X_new=np.linspace(-3, 3, 100).reshape(100, 1)
X_new_poly = poly_features.transform(X_new)
y_new = lin_reg.predict(X_new_poly)
plt.plot(X, y, "b.")
plt.plot(X_new, y_new, "r-", linewidth=2, label="Prediksi")
plt.xlabel("<span class="math-inline">x\_1</span>")
plt.ylabel("<span class="math-inline">y</span>")
plt.legend(loc="upper left")
plt.axis([-3, 3, 0, 10])
plt.title("Regresi Polinomial Derajat 2")
plt.show()
```

### a. Learning Curves

Learning curves adalah plot kinerja model pada training set dan validation set sebagai fungsi dari ukuran training set. Ini membantu mendiagnosis overfitting atau underfitting.
* Underfitting: Kedua kurva mencapai plateau, berdekatan, dan memiliki error yang tinggi.
* Overfitting: Error pada data pelatihan jauh lebih rendah daripada data validasi, dan ada celah di antara kurva.

### b. The Bias/Variance Trade-off
Error generalisasi model dapat dipecah menjadi:
* Bias: Disebabkan oleh asumsi yang salah (cenderung underfit).
* Variance: Disebabkan oleh sensitivitas berlebih terhadap data pelatihan (cenderung overfit).
* Irreducible Error: Disebabkan oleh noise pada data.

---

### 5. Model Linier yang Diregularisasi (Regularized Linear Models)

Regularisasi adalah cara untuk mengurangi overfitting dengan membatasi bobot model.

### a. Ridge Regression

Menambahkan istilah regularisasi $\ell_2$ ($\alpha \sum_{i=1}^{n} \theta_i^2$) ke fungsi biaya. Ini memaksa bobot model sekecil mungkin.

### b. Lasso Regression

Menambahkan istilah regularisasi $\ell_1$ ($\alpha \sum_{i=1}^{n} |\theta_i|$). Cenderung menghilangkan bobot fitur yang paling tidak penting (melakukan seleksi fitur otomatis).

### c. Elastic Net

Gabungan antara Ridge dan Lasso, dikendalikan oleh rasio campuran $r$.

### d. Early Stopping

Cara regularisasi dengan menghentikan pelatihan (Gradient Descent) segera setelah error validasi mencapai minimum.

```python
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Misalkan X_poly dan y dari contoh sebelumnya sudah ada

# Ridge Regression
ridge_reg = Ridge(alpha=1, solver="cholesky")
ridge_reg.fit(X_poly, y)
print("Ridge Intercept & Coef:", ridge_reg.intercept_, ridge_reg.coef_)

# Lasso Regression
lasso_reg = Lasso(alpha=0.1)
lasso_reg.fit(X_poly, y)
print("Lasso Intercept & Coef:", lasso_reg.intercept_, lasso_reg.coef_)

# Elastic Net
elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5)
elastic_net.fit(X_poly, y)
print("ElasticNet Intercept & Coef:", elastic_net.intercept_, elastic_net.coef_)
```

---

### 6. Regresi Logistik (Logistic Regression)

Digunakan untuk memperkirakan probabilitas suatu instance termasuk dalam kelas tertentu (klasifikasi biner). Model ini menerapkan fungsi logistik (sigmoid) pada hasil perhitungan bobot fitur.
Fungsi biaya yang digunakan adalah log loss (cross-entropy).

```python
from sklearn import datasets
from sklearn.linear_model import LogisticRegression

# Memuat dataset Iris
iris = datasets.load_iris()
X = iris["data"][:, 3:]  # fitur petal width
y = (iris["target"] == 2).astype(int)  # 1 jika Iris-Virginica, else 0

# Melatih model Logistic Regression
log_reg = LogisticRegression()
log_reg.fit(X, y)

# Membuat prediksi probabilitas
X_new = np.linspace(0, 3, 1000).reshape(-1, 1)
y_proba = log_reg.predict_proba(X_new)

# Plot hasil
plt.plot(X_new, y_proba[:, 1], "g-", label="Iris-Virginica")
plt.plot(X_new, y_proba[:, 0], "b--", label="Bukan Iris-Virginica")
plt.xlabel("Petal width (cm)")
plt.ylabel("Probabilitas")
plt.legend()
plt.show()

# Prediksi kelas
print("\nPrediksi untuk petal width 1.7cm:", log_reg.predict([[1.7]]))
print("Prediksi untuk petal width 1.5cm:", log_reg.predict([[1.5]]))
```

---

### 7. . Regresi Softmax (Softmax Regression)

Generalisasi dari Regresi Logistik untuk mendukung klasifikasi multikelas secara langsung. Model menghitung skor untuk setiap kelas, lalu menerapkan fungsi softmax untuk mendapatkan probabilitas. Fungsi biaya yang digunakan juga cross-entropy.

```python
X = iris["data"][:, (2, 3)] # petal length, petal width
y = iris["target"]
softmax_reg = LogisticRegression(multi_class="multinomial",solver="lbfgs", C=10)
softmax_reg.fit(X, y)

print("\nPrediksi Softmax untuk (petal length=5, petal width=2):", softmax_reg.predict([[5, 2]]))
print("Probabilitasnya:", softmax_reg.predict_proba([[5, 2]]))
```

---

### 8. Kesimpulan Bab

Bab ini berhasil menguraikan dasar-dasar model linier, termasuk Regresi Linier dan Regresi Logistik, serta bagaimana melatihnya. Pentingnya feature scaling, learning rate, dan regularisasi dibahas secara mendalam untuk mengatasi underfitting dan overfitting.